In [1]:
from datetime import datetime
from utils import utils


start_timestamp = utils.date_to_utc_timestamp(datetime(2023, 5, 23, 0, 0))
end_timestamp = utils.date_to_utc_timestamp(datetime(2023, 5, 26, 0, 0))
print(start_timestamp)
print(end_timestamp)

1684800000
1685059200


In [2]:
import pandas as pd

kucoin_df = pd.read_csv("../data/kucoin_eth_price.csv")
kucoin_df.head()

,time,high,low,open,volumefrom,volumeto,close,conversionType,conversionSymbol
0,1684800000,1818.27,1817.80,1817.80,54.66,99368.32,1818.17,direct,NaN
1,1684800060,1818.40,1818.17,1818.17,38.42,69844.82,1818.38,direct,NaN
2,1684800120,1818.48,1818.23,1818.38,30.73,55879.84,1818.41,direct,NaN
3,1684800180,1818.99,1818.41,1818.41,101.61,184832.72,1818.85,direct,NaN
4,1684800240,1818.93,1818.52,1818.85,47.14,85728.63,1818.85,direct,NaN


In [36]:
# save kucoin price weekly data
import requests


kucoin_url = "https://min-api.cryptocompare.com/data/v2/histominute?fsym=ETH&tsym=USD&limit=1000&toTs={}"

current_timestamp = end_timestamp
# current_timestamp = utils.date_to_utc_timestamp(datetime(2023, 5, 19, 1, 32))
kucoin_result = []
while current_timestamp >= start_timestamp:
    print("Start processing, current time: ", utils.utc_timestamp_to_date(current_timestamp))
    response = requests.get(kucoin_url.format(current_timestamp))
    kucoin_data = response.json()['Data']
    if not kucoin_data:
        break
    for d in kucoin_data['Data'][::-1]:
        if d["time"] >= start_timestamp:
            kucoin_result.append(d)
        else:
            break
    current_timestamp = kucoin_data['TimeFrom'] - 60

kucoin_result = kucoin_result[::-1]

print(len(kucoin_result))

Start processing, current time:  2023-05-26 00:00:00+00:00
Start processing, current time:  2023-05-25 07:19:00+00:00
Start processing, current time:  2023-05-24 14:38:00+00:00
Start processing, current time:  2023-05-23 21:57:00+00:00
Start processing, current time:  2023-05-23 05:16:00+00:00
4321


In [19]:
print(kucoin_result[0])

{'time': 1684460940, 'high': 1798.38, 'low': 1797.75, 'open': 1797.75, 'volumefrom': 57.77, 'volumeto': 103880.66, 'close': 1798.05, 'conversionType': 'direct', 'conversionSymbol': ''}


In [22]:
# ARB graphQL
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport



arb_query = """
query {
    swaps(skip: %d, orderBy: timestamp, orderDirection: asc,
        where: { pool: "0xc31e54c7a869b9fcbecc14363cf510d1c41fa443", timestamp_gte: %d }){
            id
            amountIn
            amountInUSD
            amountOut
            amountOutUSD
            tick
            timestamp
            tokenIn {
                symbol
                decimals
            }
            tokenOut {
                symbol
                decimals
            }
    }
}
"""


client = Client(
    transport=RequestsHTTPTransport(
        url='https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-arbitrum',
        verify=True,
        retries=5,
    ))

# # variables = { "num_skip": 0, "start_timestamp": 1684460940 }
# # response = client.execute(gql(arb_query), variable_values=variables)
response = client.execute(gql(arb_query % (0, start_timestamp)))
print(len(response["swaps"]))

100


In [23]:
print(response["swaps"][0])

# temp
result = []
id_set = set()

{'id': '0x13875c5e00916a733dbf870b03f29c7c0e2b575294ea3c17e3a123540b281eca0b000000', 'amountIn': '18509968', 'amountInUSD': '18.509968', 'amountOut': '10174988585088354', 'amountOutUSD': '18.50071257554249565419706', 'tick': '-201264', 'timestamp': '1684800030', 'tokenIn': {'symbol': 'USDC', 'decimals': 6}, 'tokenOut': {'symbol': 'WETH', 'decimals': 18}}


In [50]:
# response = client.execute(gql(arb_query % start_timestamp))
# result = response['swaps']
# id_set = set()
#
# for item in result:
#     id_set.add(item["id"])

skip = 0
last_cnt = 0
while int(result[-1]["timestamp"]) <= end_timestamp:
    print(datetime.now(), "result cnt:", len(result), "id cnt:", len(id_set), "skip: ", skip)
    print("Current time: ", utils.utc_timestamp_to_date(int(result[-1]["timestamp"])))
    last_cnt = len(result)
    current_timestamp = int(result[-1]["timestamp"])
    query = arb_query % (skip, current_timestamp)
    try:
        response = client.execute(gql(query))

        swap_data = response["swaps"]
        if len(swap_data) == 0:
            break
        for item in swap_data:
            if int(item["timestamp"]) > end_timestamp:
                break
            elif item["id"] in id_set:
                continue
            else:
                result.append(item)
                id_set.add(item["id"])

        if last_cnt == len(result):
            skip += 100
        else:
            skip = 0
    except Exception as ex:
        print("Fetching swap data error", ex)

2023-05-26 19:06:51.391025 result cnt: 23386 id cnt: 23386 skip:  0
Current time:  2023-05-24 13:52:26+00:00
2023-05-26 19:06:56.193864 result cnt: 23386 id cnt: 23386 skip:  100
Current time:  2023-05-24 13:52:26+00:00
2023-05-26 19:07:00.436694 result cnt: 23486 id cnt: 23486 skip:  0
Current time:  2023-05-24 13:55:08+00:00
2023-05-26 19:07:04.621134 result cnt: 23585 id cnt: 23585 skip:  0
Current time:  2023-05-24 13:58:02+00:00
2023-05-26 19:07:08.786921 result cnt: 23684 id cnt: 23684 skip:  0
Current time:  2023-05-24 14:02:25+00:00
2023-05-26 19:07:13.029153 result cnt: 23783 id cnt: 23783 skip:  0
Current time:  2023-05-24 14:06:10+00:00
2023-05-26 19:07:17.294734 result cnt: 23879 id cnt: 23879 skip:  0
Current time:  2023-05-24 14:06:43+00:00
2023-05-26 19:07:21.578973 result cnt: 23977 id cnt: 23977 skip:  0
Current time:  2023-05-24 14:08:55+00:00
2023-05-26 19:07:25.795697 result cnt: 24076 id cnt: 24076 skip:  0
Current time:  2023-05-24 14:13:49+00:00
2023-05-26 19:07:

KeyboardInterrupt: 

In [41]:
query = arb_query % current_timestamp
response = client.execute(gql(query))
swap_data = response["swaps"]
print(len(swap_data))
print(len(id_set))

100
23386


In [45]:
print(current_timestamp)
print(item["timestamp"])

1684936346
1684936346


In [46]:
print(swap_data)

[{'id': '0x12a70eac042cf435da93d9cdca04f64d4abd946270608e8f6738536fb116cba115000000', 'amountIn': '631949258', 'amountInUSD': '631.949258', 'amountOut': '351717616321909247', 'amountOutUSD': '631.8643907572813422825935', 'tick': '-201388', 'timestamp': '1684936346', 'tokenIn': {'symbol': 'USDC', 'decimals': 6}, 'tokenOut': {'symbol': 'WETH', 'decimals': 18}}, {'id': '0x71a4802f92e6d6d8f283daaaa73cf66e3bb0a87b5c58ea9ee3a2b00b237d507301020000', 'amountIn': '88000000', 'amountInUSD': '88', 'amountOut': '48975992489322807', 'amountOutUSD': '87.9858847549895606649735', 'tick': '-201388', 'timestamp': '1684936346', 'tokenIn': {'symbol': 'USDC', 'decimals': 6}, 'tokenOut': {'symbol': 'WETH', 'decimals': 18}}, {'id': '0x71a4802f92e6d6d8f283daaaa73cf66e3bb0a87b5c58ea9ee3a2b00b237d507304030000', 'amountIn': '88000000', 'amountInUSD': '88', 'amountOut': '48975345149648370', 'amountOutUSD': '87.984721802467368012885', 'tick': '-201388', 'timestamp': '1684936346', 'tokenIn': {'symbol': 'USDC', 'dec

In [44]:
time_cnt = 0
id_cnt = 0
for item in swap_data:
    if int(item["timestamp"]) > current_timestamp:
        time_cnt += 1
    elif item["id"] in id_set:
        id_cnt += 1

print(time_cnt)
print(id_cnt)

0
100


In [37]:
# save data
import pandas as pd

kucoin_df = pd.DataFrame(kucoin_result)
kucoin_df.to_csv("../../data/kucoin_eth_price.csv", index=False)

In [51]:
import pandas as pd
from copy import deepcopy


# arb_result = deepcopy(result)
arb_result1 = deepcopy(result)
for item in arb_result1:
    item["tokenIn"] = item["tokenIn"]["symbol"]
    item["tokenOut"] = item["tokenOut"]["symbol"]

arb_uniswap_df = pd.DataFrame(arb_result1)
arb_uniswap_df.to_csv("../../data/arb_swap1.csv", index=False)

In [29]:
import math

def sqrtpricex96_to_price_eth_usd(sqrtpricex96):
    """Converts a Uniswap SQRTpriceX96 to price.

    Args:
      sqrtpricex96: The SQRTpriceX96 to convert.

    Returns:
      The price.
    """
    price = sqrtpricex96 / math.pow(2, 96)
    return 1 / (price ** 2 / (10 ** 12))


print(sqrtpricex96_to_price_eth_usd(1874659030893028805438655155008306))

1786.1362752201069


In [31]:
TICK_BASE = 1.0001


def tick_to_price_eth_usd(tick):
    return 1 / (TICK_BASE ** tick / (10 ** 12))


print(tick_to_price_eth_usd(201442))

1786.1687802452334


In [36]:
def arb_tick_to_price_eth_usd(tick):
    return TICK_BASE ** tick * 10 ** 12

print(arb_tick_to_price_eth_usd(-201439))

1786.7046844661563


In [1]:
# read kucoin & arb data
import pandas as pd

kucoin_df = pd.read_csv("../../data/kucoin_eth_price.csv")
arb_df = pd.read_csv("../../data/arb_swap_all.csv")

In [2]:
arb_df.head()

,id,amountIn,amountInUSD,amountOut,amountOutUSD,tick,timestamp,tokenIn,tokenOut
0,0x13875c5e00916a733dbf870b03f29c7c0e2b575294ea...,18509968,18.509968,10174988585088354,18.500713,-201264,1684800030,USDC,WETH
1,0xa629bdac51ec7b793a04193f87006f0ff4ae910f8025...,417246407,417.246407,229361579040734245,417.037599,-201264,1684800044,USDC,WETH
2,0x38d8de741acaf8f47d3e1692926b7a663e93b6fc0e4b...,15034268999359802,27.336143,27322474,27.322474,-201264,1684800056,WETH,USDC
3,0x452dd1fbd5a7c6d03880cd92be8bc35646422b627c34...,40248724,40.248724,22124832513497315,40.228597,-201264,1684800071,USDC,WETH
4,0x7ffaad3f6e94d4e7d19df8b5f8c808512918acccf5f1...,54000000000000000,98.185802,98136699,98.136699,-201264,1684800077,WETH,USDC


In [3]:
kucoin_df.head()

,time,high,low,open,volumefrom,volumeto,close,conversionType,conversionSymbol
0,1684800000,1818.27,1817.80,1817.80,54.66,99368.32,1818.17,direct,NaN
1,1684800060,1818.40,1818.17,1818.17,38.42,69844.82,1818.38,direct,NaN
2,1684800120,1818.48,1818.23,1818.38,30.73,55879.84,1818.41,direct,NaN
3,1684800180,1818.99,1818.41,1818.41,101.61,184832.72,1818.85,direct,NaN
4,1684800240,1818.93,1818.52,1818.85,47.14,85728.63,1818.85,direct,NaN


In [6]:
# convert arb swap data to minute price
def get_trade_data(swap_dict):
    if swap_dict["tokenIn"] == "WETH":
        eth_amount = int(swap_dict["amountIn"]) / (10 ** 18)
        usd = float(swap_dict["amountInUSD"])
    else:
        eth_amount = int(swap_dict["amountOut"]) / (10 ** 18)
        usd = float(swap_dict["amountOutUSD"])

    if eth_amount != 0:
        return usd / eth_amount, usd
    return 0, usd


arb_result = arb_df.to_dict("records")

error_result = []
new_result = []
for r in arb_result:
    price, trade_usd = get_trade_data(r)
    if price == 0:
        error_result.append(r)
    else:
        r["price"] = price
        r["trade_usd"] = trade_usd
        new_result.append(r)


print(len(error_result))
print(len(new_result))
print(new_result[0])

2
46367


In [8]:
kucoin_time_list = kucoin_df["time"].tolist()[::-1]

def get_time_key(timestamp):
    end_time = kucoin_time_list[0] + 60

    for t in kucoin_time_list:
        if t < timestamp <= end_time:
            return t
    return None


minute_price_dict = dict()

for r in new_result[::-1]:
    swap_time = r["timestamp"]
    time_key = get_time_key(swap_time)
    if time_key not in minute_price_dict:
        minute_price_dict[time_key] = [(r["price"], r["trade_usd"])]
    elif time_key is not None:
        minute_price_dict[time_key].append((r["price"], r["trade_usd"]))


# stats minute price
print(len(new_result))
print(len(minute_price_dict.keys()))
print(list(minute_price_dict.keys())[:10])

46367
4272
[1685059080, 1685059020, 1685058960, 1685058900, 1685058840, 1685058780, 1685058720, 1685058660, 1685058540, 1685058480]


In [9]:
import pandas as pd


def get_price_detail(trade_list):
    price_detail_dict = dict()
    price_list = [p[0] for p in trade_list]
    price_detail_dict["open"] = price_list[-1]
    price_detail_dict["close"] = price_list[0]
    price_detail_dict["high"] = max(price_list)
    price_detail_dict["low"] = min(price_list)

    usd_list = [p[1] for p in trade_list]
    price_detail_dict["volume"] = sum(usd_list)
    return price_detail_dict


minute_stats = []
for time_key in sorted(minute_price_dict.keys()):
    stats_dict = get_price_detail(minute_price_dict[time_key])
    stats_dict["time"] = time_key
    minute_stats.append(stats_dict)


arb_df = pd.DataFrame(minute_stats)
arb_df.head()

,open,close,high,low,volume,time
0,1818.25389,1818.25555,1818.25555,1818.25389,462.874455,1684800000
1,1818.25545,1818.25784,1818.25784,1818.25545,2996.957906,1684800060
2,1818.24957,1818.37415,1818.37415,1818.24957,32740.669877,1684800180
3,1818.37420,1818.37420,1818.37420,1818.37420,55.016836,1684800240
4,1818.37399,1818.37403,1818.37403,1818.37399,32740.751811,1684800300


In [11]:
# arb_df join kucoin_df by time schema and left high, low schema
df = pd.merge(arb_df, kucoin_df, on='time')[["time", "high_x", "low_x", "high_y", "low_y", "volume", "volumeto"]]
df.head()

,time,high_x,low_x,high_y,low_y,volume,volumeto
0,1684800000,1818.25555,1818.25389,1818.27,1817.80,462.874455,99368.32
1,1684800060,1818.25784,1818.25545,1818.40,1818.17,2996.957906,69844.82
2,1684800180,1818.37415,1818.24957,1818.99,1818.41,32740.669877,184832.72
3,1684800240,1818.37420,1818.37420,1818.93,1818.52,55.016836,85728.63
4,1684800300,1818.37403,1818.37399,1819.17,1818.85,32740.751811,49105.83


In [12]:
df.rename(columns={
    "high_x": "arb_high",
    "low_x": "arb_low",
    "high_y": "kucoin_high",
    "low_y": "kucoin_low",
    "volume": "arb_volume",
    "volumeto": "kucoin_volume",
}, inplace=True)

df.head()

,time,arb_high,arb_low,kucoin_high,kucoin_low,arb_volume,kucoin_volume
0,1684800000,1818.25555,1818.25389,1818.27,1817.80,462.874455,99368.32
1,1684800060,1818.25784,1818.25545,1818.40,1818.17,2996.957906,69844.82
2,1684800180,1818.37415,1818.24957,1818.99,1818.41,32740.669877,184832.72
3,1684800240,1818.37420,1818.37420,1818.93,1818.52,55.016836,85728.63
4,1684800300,1818.37403,1818.37399,1819.17,1818.85,32740.751811,49105.83


In [13]:
df["high_diff"] = df["arb_high"] - df["kucoin_high"]
df["high_diff_rate"] = abs(df["high_diff"]) / df["arb_high"]
df["low_diff"] = df["arb_low"] - df["kucoin_low"]
df["low_diff_rate"] = abs(df["low_diff"]) / df["arb_low"]

df.head()

,time,arb_high,arb_low,kucoin_high,kucoin_low,arb_volume,kucoin_volume,high_diff,high_diff_rate,low_diff,low_diff_rate
0,1684800000,1818.25555,1818.25389,1818.27,1817.80,462.874455,99368.32,-0.01445,0.000008,0.45389,0.000250
1,1684800060,1818.25784,1818.25545,1818.40,1818.17,2996.957906,69844.82,-0.14216,0.000078,0.08545,0.000047
2,1684800180,1818.37415,1818.24957,1818.99,1818.41,32740.669877,184832.72,-0.61585,0.000339,-0.16043,0.000088
3,1684800240,1818.37420,1818.37420,1818.93,1818.52,55.016836,85728.63,-0.55580,0.000306,-0.14580,0.000080
4,1684800300,1818.37403,1818.37399,1819.17,1818.85,32740.751811,49105.83,-0.79597,0.000438,-0.47601,0.000262


In [45]:
df.sort_values(by=["high_diff"], ascending=False, inplace=True)
df.head(50)

,time,arb_high,arb_low,kucoin_high,kucoin_low,arb_volume,kucoin_volume,high_diff,high_diff_rate,low_diff,low_diff_rate,kucoin_gap,arb_gap
2931,1684978020,1770.01292,1767.47280,1767.34,1764.28,4.514917e+05,2724207.96,2.67292,0.001510,3.19280,0.001806,3.06,2.54012
1629,1684899360,1825.65451,1820.21040,1823.10,1813.17,1.119969e+06,5077359.02,2.55451,0.001399,7.04040,0.003868,9.93,5.44411
1635,1684899720,1820.02112,1819.42965,1817.87,1816.45,1.223121e+05,1008461.31,2.15112,0.001182,2.97965,0.001638,1.42,0.59147
2932,1684978080,1769.04036,1767.46773,1766.89,1763.39,4.842366e+05,4177774.37,2.15036,0.001216,4.07773,0.002307,3.50,1.57263
1634,1684899660,1820.28375,1820.05236,1818.35,1816.93,5.318695e+04,599747.13,1.93375,0.001062,3.12236,0.001716,1.42,0.23139
2260,1684937220,1788.10564,1785.44609,1786.28,1782.57,4.616557e+05,1628276.30,1.82564,0.001021,2.87609,0.001611,3.71,2.65955
2930,1684977960,1781.39048,1769.13005,1779.60,1761.75,1.738033e+06,12016874.02,1.79048,0.001005,7.38005,0.004172,17.85,12.26043
1631,1684899480,1821.00177,1820.73947,1819.30,1817.37,7.217439e+04,502431.15,1.70177,0.000935,3.36947,0.001851,1.93,0.26230
389,1684823640,1857.77543,1857.76906,1856.13,1855.59,1.247590e+03,140135.87,1.64543,0.000886,2.17906,0.001173,0.54,0.00637
1658,1684901100,1820.84445,1820.47905,1819.21,1818.70,1.115798e+05,162363.48,1.63445,0.000898,1.77905,0.000977,0.51,0.36540


In [46]:
df.sort_values(by=["low_diff"], ascending=False, inplace=True)
df.head(50)

,time,arb_high,arb_low,kucoin_high,kucoin_low,arb_volume,kucoin_volume,high_diff,high_diff_rate,low_diff,low_diff_rate,kucoin_gap,arb_gap
2930,1684977960,1781.39048,1769.13005,1779.60,1761.75,1.738033e+06,12016874.02,1.79048,0.001005,7.38005,0.004172,17.85,12.26043
2259,1684937160,1794.26047,1785.04584,1793.17,1777.98,1.326263e+06,8931168.39,1.09047,0.000608,7.06584,0.003958,15.19,9.21463
1629,1684899360,1825.65451,1820.21040,1823.10,1813.17,1.119969e+06,5077359.02,2.55451,0.001399,7.04040,0.003868,9.93,5.44411
2245,1684936320,1804.98630,1795.92821,1803.49,1790.39,2.635538e+06,6926729.09,1.49630,0.000829,5.53821,0.003084,13.10,9.05809
2932,1684978080,1769.04036,1767.46773,1766.89,1763.39,4.842366e+05,4177774.37,2.15036,0.001216,4.07773,0.002307,3.50,1.57263
1630,1684899420,1821.08974,1820.39379,1820.27,1816.49,1.620217e+05,1437353.56,0.81974,0.000450,3.90379,0.002144,3.78,0.69595
1628,1684899300,1828.99055,1825.65451,1828.91,1821.92,8.734903e+05,2915011.75,0.08055,0.000044,3.73451,0.002046,6.99,3.33604
1631,1684899480,1821.00177,1820.73947,1819.30,1817.37,7.217439e+04,502431.15,1.70177,0.000935,3.36947,0.001851,1.93,0.26230
2931,1684978020,1770.01292,1767.47280,1767.34,1764.28,4.514917e+05,2724207.96,2.67292,0.001510,3.19280,0.001806,3.06,2.54012
2316,1684940580,1785.71129,1784.50591,1784.22,1781.32,2.187805e+05,1459611.23,1.49129,0.000835,3.18591,0.001785,2.90,1.20538


In [19]:
minute_price_dict[1684977960]

[(1769.4293599999999, 6626.393923568697),
 (1769.4293599999999, 250.3699255439876),
 (1769.13005, 31844.3409),
 (1769.13005, 100.94920742282656),
 (1769.13005, 100.94926054685985),
 (1769.1300499999998, 100.94931367093506),
 (1769.13005, 100.94936679505221),
 (1769.1300499999998, 100.9494199192113),
 (1769.13005, 100.94947304341233),
 (1769.6466400000002, 56046.832664768),
 (1769.6527699999997, 664.4461800418076),
 (1769.65071, 223.4340346378958),
 (1769.73482, 1859.2483910182832),
 (1769.7348200000001, 4067.9090656855537),
 (1769.7348200000001, 6916.182520152976),
 (1769.80622, 227.570417136639),
 (1769.8062200000002, 7972.463981541213),
 (1770.84708, 5699.327669312736),
 (1770.84708, 124.198574499944),
 (1770.8470799999998, 262.81950651685935),
 (1770.8470800000002, 166.0443052948351),
 (1770.84708, 716.0580468532855),
 (1770.8470799999998, 1280.009460957504),
 (1770.84708, 70.00509938215207),
 (1770.8470799999998, 107.75681982348422),
 (1770.8470799999998, 132.3170718961055),
 (1770

In [26]:
# price fast change period
from utils import utils
from datetime import datetime

print(utils.date_to_utc_timestamp(datetime(2023, 5, 23, 1, 0)))

1684803600


In [24]:
df.sort_values(by=["time"], ascending=True, inplace=True)
df.head(10)

,time,arb_high,arb_low,kucoin_high,kucoin_low,arb_volume,kucoin_volume,high_diff,high_diff_rate,low_diff,low_diff_rate
0,1684800000,1818.25555,1818.25389,1818.27,1817.80,462.874455,99368.32,-0.01445,0.000008,0.45389,0.000250
1,1684800060,1818.25784,1818.25545,1818.40,1818.17,2996.957906,69844.82,-0.14216,0.000078,0.08545,0.000047
2,1684800180,1818.37415,1818.24957,1818.99,1818.41,32740.669877,184832.72,-0.61585,0.000339,-0.16043,0.000088
3,1684800240,1818.37420,1818.37420,1818.93,1818.52,55.016836,85728.63,-0.55580,0.000306,-0.14580,0.000080
4,1684800300,1818.37403,1818.37399,1819.17,1818.85,32740.751811,49105.83,-0.79597,0.000438,-0.47601,0.000262
5,1684800360,1818.63848,1818.49863,1819.44,1819.13,36735.875500,104732.32,-0.80152,0.000441,-0.63137,0.000347
6,1684800420,1818.64388,1818.63848,1819.45,1819.26,3237.218458,61450.85,-0.80612,0.000443,-0.62152,0.000342
7,1684800480,1818.64111,1818.64111,1819.43,1819.33,214.860556,21242.51,-0.78889,0.000434,-0.68889,0.000379
8,1684800540,1818.64193,1818.64193,1819.43,1819.29,4.116292,13132.65,-0.78807,0.000433,-0.64807,0.000356
9,1684800720,1818.64195,1818.64195,1819.33,1819.16,3443.965077,57162.59,-0.68805,0.000378,-0.51805,0.000285


In [30]:
df[df["time"] >= 1684805340].head(200)

,time,arb_high,arb_low,kucoin_high,kucoin_low,arb_volume,kucoin_volume,high_diff,high_diff_rate,low_diff,low_diff_rate
84,1684805340,1832.09506,1823.41922,1832.58,1823.74,2.627307e+06,6600945.07,-0.48494,0.000265,-0.32078,0.000176
85,1684805400,1839.50176,1831.78591,1841.35,1831.74,2.081068e+06,3721682.26,-1.84824,0.001005,0.04591,0.000025
86,1684805460,1840.27585,1838.37083,1841.61,1839.12,4.872914e+05,1233587.28,-1.33415,0.000725,-0.74917,0.000408
87,1684805520,1839.52977,1838.98115,1840.15,1839.06,1.885188e+05,2182820.27,-0.62023,0.000337,-0.07885,0.000043
88,1684805580,1838.84391,1838.32013,1840.19,1838.28,9.366620e+04,406585.13,-1.34609,0.000732,0.04013,0.000022
...,...,...,...,...,...,...,...,...,...,...,...
279,1684817040,1858.04763,1856.92268,1858.45,1856.87,1.759482e+05,517803.58,-0.40237,0.000217,0.05268,0.000028
280,1684817100,1859.03868,1857.96531,1859.09,1857.85,1.807542e+05,264824.06,-0.05132,0.000028,0.11531,0.000062
281,1684817160,1859.03958,1859.02414,1859.12,1858.31,3.933063e+03,398156.13,-0.08042,0.000043,0.71414,0.000384
282,1684817220,1859.02429,1859.01766,1859.07,1858.36,1.322605e+03,118311.25,-0.04571,0.000025,0.65766,0.000354


In [28]:
kucoin_df[kucoin_df["time"] == 1684805340]

,time,high,low,open,volumefrom,volumeto,close,conversionType,conversionSymbol
89,1684805340,1832.58,1823.74,1824.13,3608.9,6600945.07,1831.74,direct,NaN


In [32]:
# CEX & DEX trading volume high
df["kucoin_gap"] = df["kucoin_high"] - df["kucoin_low"]
df["arb_gap"] = df["arb_high"] - df["arb_low"]

df.sort_values(by=["kucoin_volume"], ascending=False, inplace=True)
df.head(20)

,time,arb_high,arb_low,kucoin_high,kucoin_low,arb_volume,kucoin_volume,high_diff,high_diff_rate,low_diff,low_diff_rate,kucoin_gap,arb_gap
2930,1684977960,1781.39048,1769.13005,1779.60,1761.75,1.738033e+06,12016874.02,1.79048,0.001005,7.38005,0.004172,17.85,12.26043
195,1684812000,1867.37294,1852.71090,1870.55,1853.98,3.894675e+06,9282686.39,-3.17706,0.001701,-1.26910,0.000685,16.57,14.66204
2259,1684937160,1794.26047,1785.04584,1793.17,1777.98,1.326263e+06,8931168.39,1.09047,0.000608,7.06584,0.003958,15.19,9.21463
3567,1685016300,1807.34627,1792.56992,1807.50,1793.52,4.259063e+06,8462290.52,-0.15373,0.000085,-0.95008,0.000530,13.98,14.77635
2245,1684936320,1804.98630,1795.92821,1803.49,1790.39,2.635538e+06,6926729.09,1.49630,0.000829,5.53821,0.003084,13.10,9.05809
84,1684805340,1832.09506,1823.41922,1832.58,1823.74,2.627307e+06,6600945.07,-0.48494,0.000265,-0.32078,0.000176,8.84,8.67584
193,1684811880,1852.58689,1846.62397,1854.52,1846.01,1.121430e+06,5445993.10,-1.93311,0.001043,0.61397,0.000332,8.51,5.96292
1629,1684899360,1825.65451,1820.21040,1823.10,1813.17,1.119969e+06,5077359.02,2.55451,0.001399,7.04040,0.003868,9.93,5.44411
2929,1684977900,1787.41563,1781.39048,1787.56,1778.58,1.151800e+06,4949631.42,-0.14437,0.000081,2.81048,0.001578,8.98,6.02515
2932,1684978080,1769.04036,1767.46773,1766.89,1763.39,4.842366e+05,4177774.37,2.15036,0.001216,4.07773,0.002307,3.50,1.57263


In [33]:
df.sort_values(by=["kucoin_gap"], ascending=False, inplace=True)
df.head(20)

,time,arb_high,arb_low,kucoin_high,kucoin_low,arb_volume,kucoin_volume,high_diff,high_diff_rate,low_diff,low_diff_rate,kucoin_gap,arb_gap
2930,1684977960,1781.39048,1769.13005,1779.60,1761.75,1.738033e+06,12016874.02,1.79048,0.001005,7.38005,0.004172,17.85,12.26043
195,1684812000,1867.37294,1852.71090,1870.55,1853.98,3.894675e+06,9282686.39,-3.17706,0.001701,-1.26910,0.000685,16.57,14.66204
2259,1684937160,1794.26047,1785.04584,1793.17,1777.98,1.326263e+06,8931168.39,1.09047,0.000608,7.06584,0.003958,15.19,9.21463
3567,1685016300,1807.34627,1792.56992,1807.50,1793.52,4.259063e+06,8462290.52,-0.15373,0.000085,-0.95008,0.000530,13.98,14.77635
2245,1684936320,1804.98630,1795.92821,1803.49,1790.39,2.635538e+06,6926729.09,1.49630,0.000829,5.53821,0.003084,13.10,9.05809
1629,1684899360,1825.65451,1820.21040,1823.10,1813.17,1.119969e+06,5077359.02,2.55451,0.001399,7.04040,0.003868,9.93,5.44411
85,1684805400,1839.50176,1831.78591,1841.35,1831.74,2.081068e+06,3721682.26,-1.84824,0.001005,0.04591,0.000025,9.61,7.71585
2929,1684977900,1787.41563,1781.39048,1787.56,1778.58,1.151800e+06,4949631.42,-0.14437,0.000081,2.81048,0.001578,8.98,6.02515
84,1684805340,1832.09506,1823.41922,1832.58,1823.74,2.627307e+06,6600945.07,-0.48494,0.000265,-0.32078,0.000176,8.84,8.67584
193,1684811880,1852.58689,1846.62397,1854.52,1846.01,1.121430e+06,5445993.10,-1.93311,0.001043,0.61397,0.000332,8.51,5.96292


In [36]:
# compare kucoin volume with arb uniswap volume
print(df["kucoin_volume"].sum())
print(df["arb_volume"].sum())
print(df["kucoin_volume"].sum() / df["arb_volume"].sum())

912497906.41
270864581.65722555
3.368834348245465


In [48]:
# when price change rapidly, analysis the trading pattern
specific_time = 1684812000

timekey_data = []
for r in new_result[::-1]:
    swap_time = r["timestamp"]
    time_key = get_time_key(swap_time)
    if time_key != specific_time:
        continue
    timekey_data.append(r)


test_df = pd.DataFrame(timekey_data)
test_df.sort_values(by=["trade_usd"], ascending=False, inplace=True)
test_df.head(200)

,id,amountIn,amountInUSD,amountOut,amountOutUSD,tick,timestamp,tokenIn,tokenOut,price,trade_usd
0,0xc14484b7a4f3d744155e1c6ee1e0c2bad0445c2bafd7...,16804947000,16804.947000,8994553005199080039,16787.791442,-200997,1684812060,USDC,WETH,1866.43977,16787.791442
1,0xae9e5c4e377e4b69f09ef00d826ac65d82944e927edb...,13907930153,13907.930153,7443948839151860718,13900.628629,-200997,1684812060,USDC,WETH,1867.37294,13900.628629
2,0x771f313ea2af73261f6675ff08731f7e70b05b6c81ff...,9721857323502363978,18145.261147,18145848341,18145.848341,-200998,1684812060,WETH,USDC,1866.43977,18145.261147
3,0x6a51945d54b7ab21a160e54806d04b11ad84ce9c48e7...,90368556932,90368.556932,48358945911922369158,90304.187003,-200994,1684812060,USDC,WETH,1867.37294,90304.187003
4,0x67bb71b3aa630ae778210525b7aab229d7b3bdb3e459...,1380424126525724672,2576.478489,2576467050,2576.467050,-200998,1684812060,WETH,USDC,1866.43977,2576.478489
...,...,...,...,...,...,...,...,...,...,...,...
195,0xb6203ba5fe848af9ff361cc96bbf768eff29e43c2b04...,18237141178,18237.141178,9827066718439991672,18225.809384,-201065,1684812011,USDC,WETH,1854.65408,18225.809384
196,0x63023f27fa5c68b9f5e47d90ca39f2c2732e577091bb...,33407915831,33407.915831,18000000000000000000,33383.773440,-201063,1684812011,USDC,WETH,1854.65408,33383.773440
197,0x3b8ca796e74430509819ec77b20683a885f2fff2555e...,228013916015625010,422.886940,422802026,422.802026,-201063,1684812011,WETH,USDC,1854.65408,422.886940
198,0x064713f15881a6d0e32aa4a7a9246074ca0be4d5469a...,23316255162,23316.255162,12564967395071314732,23303.668044,-201065,1684812010,USDC,WETH,1854.65408,23303.668044


In [50]:
df[df["time"] == specific_time].head()

,time,arb_high,arb_low,kucoin_high,kucoin_low,arb_volume,kucoin_volume,high_diff,high_diff_rate,low_diff,low_diff_rate,kucoin_gap,arb_gap
195,1684812000,1867.37294,1852.7109,1870.55,1853.98,3.894675e+06,9282686.39,-3.17706,0.001701,-1.2691,0.000685,16.57,14.66204


In [1]:
# when price change rapidly, who is trading in uniswap
# get gas price, close time to rapid price change

import pandas as pd

analysis_df = pd.read_csv("../data/arb_swap_0523_0528.csv")
analysis_df.head()

,id,gasLimit,gasPrice,amountIn,amountInUSD,amountOut,amountOutUSD,tick,timestamp,tokenIn,tokenOut,transaction,trader,traderPositionCount,traderOpenPositionCount,traderClosedPositionCount,traderDepositCount,traderWithdrawCount,traderSwapCount
0,0x13875c5e00916a733dbf870b03f29c7c0e2b575294ea...,8000000,400000000,18509968,18.509968,10174988585088354,18.500713,-201264,1684800030,USDC,WETH,0x13875c5e00916a733dbf870b03f29c7c0e2b575294ea...,0x112bf8e7431bde9bd9395d44307f788a6e054691,0,0,0,3695,3695,3695
1,0xa629bdac51ec7b793a04193f87006f0ff4ae910f8025...,28410397,200000000,417246407,417.246407,229361579040734245,417.037599,-201264,1684800044,USDC,WETH,0xa629bdac51ec7b793a04193f87006f0ff4ae910f8025...,0x59c5c9045374693b384bc023fdca4685c7ec1a9e,0,0,0,5046,5046,5046
2,0x38d8de741acaf8f47d3e1692926b7a663e93b6fc0e4b...,2589363,100000000,15034268999359802,27.336143,27322474,27.322474,-201264,1684800056,WETH,USDC,0x38d8de741acaf8f47d3e1692926b7a663e93b6fc0e4b...,0x679d1c9f327257c4cec9ebc1c7c750728351e989,0,0,0,410,410,410
3,0x452dd1fbd5a7c6d03880cd92be8bc35646422b627c34...,6430718,100000000,40248724,40.248724,22124832513497315,40.228597,-201264,1684800071,USDC,WETH,0x452dd1fbd5a7c6d03880cd92be8bc35646422b627c34...,0x2b185c762f6fb49a07116ec06b238b58fe2bec53,3,3,0,293,297,290
4,0x7ffaad3f6e94d4e7d19df8b5f8c808512918acccf5f1...,2526058,150000000,54000000000000000,98.185802,98136699,98.136699,-201264,1684800077,WETH,USDC,0x7ffaad3f6e94d4e7d19df8b5f8c808512918acccf5f1...,0x23e08e6e721a0a56989c8759afa1841cd07e9565,0,0,0,4,4,4


In [2]:
analysis_df.columns

Index(['id', 'gasLimit', 'gasPrice', 'amountIn', 'amountInUSD', 'amountOut',
       'amountOutUSD', 'tick', 'timestamp', 'tokenIn', 'tokenOut',
       'transaction', 'trader', 'traderPositionCount',
       'traderOpenPositionCount', 'traderClosedPositionCount',
       'traderDepositCount', 'traderWithdrawCount', 'traderSwapCount'],
      dtype='object')

In [6]:
df = analysis_df[["id", "timestamp", "gasLimit", "gasPrice", "amountIn", "amountInUSD", "amountOut", "amountOutUSD", "tick", "tokenIn", "tokenOut", "trader", "traderSwapCount"]]
df.head()

,id,timestamp,gasLimit,gasPrice,amountIn,amountInUSD,amountOut,amountOutUSD,tick,tokenIn,tokenOut,trader,traderSwapCount
0,0x13875c5e00916a733dbf870b03f29c7c0e2b575294ea...,1684800030,8000000,400000000,18509968,18.509968,10174988585088354,18.500713,-201264,USDC,WETH,0x112bf8e7431bde9bd9395d44307f788a6e054691,3695
1,0xa629bdac51ec7b793a04193f87006f0ff4ae910f8025...,1684800044,28410397,200000000,417246407,417.246407,229361579040734245,417.037599,-201264,USDC,WETH,0x59c5c9045374693b384bc023fdca4685c7ec1a9e,5046
2,0x38d8de741acaf8f47d3e1692926b7a663e93b6fc0e4b...,1684800056,2589363,100000000,15034268999359802,27.336143,27322474,27.322474,-201264,WETH,USDC,0x679d1c9f327257c4cec9ebc1c7c750728351e989,410
3,0x452dd1fbd5a7c6d03880cd92be8bc35646422b627c34...,1684800071,6430718,100000000,40248724,40.248724,22124832513497315,40.228597,-201264,USDC,WETH,0x2b185c762f6fb49a07116ec06b238b58fe2bec53,290
4,0x7ffaad3f6e94d4e7d19df8b5f8c808512918acccf5f1...,1684800077,2526058,150000000,54000000000000000,98.185802,98136699,98.136699,-201264,WETH,USDC,0x23e08e6e721a0a56989c8759afa1841cd07e9565,4


In [7]:
df.shape

(68838, 13)

In [8]:
df.describe()

,timestamp,gasLimit,gasPrice,amountInUSD,amountOutUSD,tick,traderSwapCount
count,6.883800e+04,6.883800e+04,6.883800e+04,6.883800e+04,68838.000000,68838.000000,68838.000000
mean,1.684997e+09,6.849995e+06,3.526308e+08,5.545148e+03,5541.608213,-201260.433627,24097.274993
std,1.192087e+05,3.070028e+07,2.511139e+09,1.270224e+04,12690.971837,129.618763,62154.914210
min,1.684800e+09,4.881170e+05,1.000000e+08,1.700000e-15,0.000000,-201553.000000,1.000000
25%,1.684901e+09,2.160670e+06,1.000000e+08,4.001971e+01,40.000000,-201360.000000,19.000000
50%,1.684993e+09,3.239952e+06,1.000000e+08,6.980816e+02,697.737242,-201260.000000,711.000000
75%,1.685095e+09,6.745376e+06,2.000000e+08,5.595078e+03,5592.228392,-201178.000000,9957.000000
max,1.685232e+09,2.000000e+09,1.000000e+11,9.256234e+05,923623.475197,-200979.000000,335454.000000


### analysis：
###   1. trading account analysis
###   2. trading volume in and before price rapid change period

In [9]:
### only left trade usd larger than 1000, since smaller swap may not be able to get enough profit
df = df[df["amountOutUSD"] > 1000]
df.shape

(32139, 13)

In [3]:
# considering trading fee, calculate "0x8cc02c2381b7c55e18dccfea917f0677a5671931" invest, profit and positions

df = pd.read_csv("../data/arb_swap.csv")
df.head()

,gasLimit,gasPrice,amountIn,amountInUSD,amountOut,amountOutUSD,tick,timestamp,tokenIn,tokenOut,...,trader,traderPositionCount,traderOpenPositionCount,traderClosedPositionCount,traderDepositCount,traderWithdrawCount,traderSwapCount,ethAmount,usdAmount,price
0,8000000,400000000,18509968,18.509968,10174988585088354,18.500713,-201264,1684800030,USDC,WETH,...,0x112bf8e7431bde9bd9395d44307f788a6e054691,0,0,0,3695,3695,3695,0.010175,18.500713,1818.25389
1,28410397,200000000,417246407,417.246407,229361579040734245,417.037599,-201264,1684800044,USDC,WETH,...,0x59c5c9045374693b384bc023fdca4685c7ec1a9e,0,0,0,5046,5046,5046,0.229362,417.037599,1818.25396
2,2589363,100000000,15034268999359802,27.336143,27322474,27.322474,-201264,1684800056,WETH,USDC,...,0x679d1c9f327257c4cec9ebc1c7c750728351e989,0,0,0,410,410,410,0.015034,27.336143,1818.25555
3,6430718,100000000,40248724,40.248724,22124832513497315,40.228597,-201264,1684800071,USDC,WETH,...,0x2b185c762f6fb49a07116ec06b238b58fe2bec53,3,3,0,293,297,290,0.022125,40.228597,1818.25545
4,2526058,150000000,54000000000000000,98.185802,98136699,98.136699,-201264,1684800077,WETH,USDC,...,0x23e08e6e721a0a56989c8759afa1841cd07e9565,0,0,0,4,4,4,0.054000,98.185802,1818.25560


In [5]:
user_df = df[df["trader"] == "0x8cc02c2381b7c55e18dccfea917f0677a5671931"]
user_df.shape

(1989, 21)

In [6]:
user_df.head()

,gasLimit,gasPrice,amountIn,amountInUSD,amountOut,amountOutUSD,tick,timestamp,tokenIn,tokenOut,...,trader,traderPositionCount,traderOpenPositionCount,traderClosedPositionCount,traderDepositCount,traderWithdrawCount,traderSwapCount,ethAmount,usdAmount,price
44,6000000,2000000000,28963484807,28963.484807,15916883943069540192,28948.125166,-201261,1684801307,USDC,WETH,...,0x8cc02c2381b7c55e18dccfea917f0677a5671931,0,0,0,60818,60818,60818,15.916884,28948.125166,1818.70555
70,6000000,2000000000,26561134416,26561.134416,14592424671772653034,26544.819975,-201258,1684801781,USDC,WETH,...,0x8cc02c2381b7c55e18dccfea917f0677a5671931,0,0,0,60818,60818,60818,14.592425,26544.819975,1819.08220
110,6000000,2000000000,23734455145,23734.455145,13026029637798678527,23721.125260,-201248,1684802011,USDC,WETH,...,0x8cc02c2381b7c55e18dccfea917f0677a5671931,0,0,0,60818,60818,60818,13.026030,23721.125260,1821.05568
121,6000000,2000000000,37540012512,37540.012512,20597925075725121430,37518.016271,-201246,1684802117,USDC,WETH,...,0x8cc02c2381b7c55e18dccfea917f0677a5671931,0,0,0,60818,60818,60818,20.597925,37518.016271,1821.44639
167,6000000,2000000000,14727537731424333731,26848.252389,26833703476,26833.703476,-201239,1684802251,WETH,USDC,...,0x8cc02c2381b7c55e18dccfea917f0677a5671931,0,0,0,60818,60818,60818,14.727538,26848.252389,1822.99668


In [11]:
init_eth = 100
init_usd = 100 * 2000

# calculate eth, usd in every step to know least investment, considering about trading fee
trade_data = user_df.to_dict("records")
trade_data[0]

{'gasLimit': 6000000,
 'gasPrice': 2000000000,
 'amountIn': '28963484807',
 'amountInUSD': 28963.484807,
 'amountOut': '15916883943069540192',
 'amountOutUSD': 28948.125165966456,
 'tick': -201261,
 'timestamp': 1684801307,
 'tokenIn': 'USDC',
 'tokenOut': 'WETH',
 'transaction': '0x727d427aeccdb9ff40de206105fcfbe9780d071fa9aa5081a283ccae7dd70a8403000000',
 'trader': '0x8cc02c2381b7c55e18dccfea917f0677a5671931',
 'traderPositionCount': 0,
 'traderOpenPositionCount': 0,
 'traderClosedPositionCount': 0,
 'traderDepositCount': 60818,
 'traderWithdrawCount': 60818,
 'traderSwapCount': 60818,
 'ethAmount': 15.91688394306954,
 'usdAmount': 28948.125165966456,
 'price': 1818.70555,
 'fees': 14.4817424035,
 'usd': 71036.515193,
 'eth': 115.91688394306954}

In [18]:
last_usd = init_usd
last_eth = init_eth

current_usd = 0
current_eth = 0
for item in trade_data:
    fees = item["amountInUSD"] * 0.0005
    if item["tokenIn"] == "USDC":
        current_usd = last_usd - item["amountInUSD"]
        current_eth = last_eth + int(item["amountOut"]) / (10 ** 18)
    if item["tokenIn"] == "WETH":
        current_usd = last_usd + item["amountOutUSD"]
        current_eth = last_eth - int(item["amountIn"]) / (10 ** 18)
    profit = item["price"] * current_eth + current_usd - item["price"] * init_eth - init_usd
    item["profit"] = profit
    item["fees"] = fees
    item["usd"] = current_usd
    item["eth"] = current_eth
    last_usd = current_usd
    last_eth = current_eth

user_df = pd.DataFrame(trade_data)
user_df.head()

,gasLimit,gasPrice,amountIn,amountInUSD,amountOut,amountOutUSD,tick,timestamp,tokenIn,tokenOut,...,traderDepositCount,traderWithdrawCount,traderSwapCount,ethAmount,usdAmount,price,fees,usd,eth,profit
0,6000000,2000000000,28963484807,28963.484807,15916883943069540192,28948.125166,-201261,1684801307,USDC,WETH,...,60818,60818,60818,15.916884,28948.125166,1818.70555,14.481742,171036.515193,115.916884,-15.359641
1,6000000,2000000000,26561134416,26561.134416,14592424671772653034,26544.819975,-201258,1684801781,USDC,WETH,...,60818,60818,60818,14.592425,26544.819975,1819.08220,13.280567,144475.380777,130.509309,-25.678987
2,6000000,2000000000,23734455145,23734.455145,13026029637798678527,23721.125260,-201248,1684802011,USDC,WETH,...,60818,60818,60818,13.026030,23721.125260,1821.05568,11.867228,120740.925632,143.535338,21.200638
3,6000000,2000000000,37540012512,37540.012512,20597925075725121430,37518.016271,-201246,1684802117,USDC,WETH,...,60818,60818,60818,20.597925,37518.016271,1821.44639,18.770006,83200.913120,164.133263,16.214088
4,6000000,2000000000,14727537731424333731,26848.252389,26833703476,26833.703476,-201239,1684802251,WETH,USDC,...,60818,60818,60818,14.727538,26848.252389,1822.99668,13.424126,110034.616596,149.405726,101.090332


In [13]:
user_df.columns

Index(['gasLimit', 'gasPrice', 'amountIn', 'amountInUSD', 'amountOut',
       'amountOutUSD', 'tick', 'timestamp', 'tokenIn', 'tokenOut',
       'transaction', 'trader', 'traderPositionCount',
       'traderOpenPositionCount', 'traderClosedPositionCount',
       'traderDepositCount', 'traderWithdrawCount', 'traderSwapCount',
       'ethAmount', 'usdAmount', 'price', 'fees', 'usd', 'eth'],
      dtype='object')

In [19]:
user_df["datetime"] = user_df["timestamp"].apply(utils.utc_timestamp_to_date)
user_df.head()

,gasLimit,gasPrice,amountIn,amountInUSD,amountOut,amountOutUSD,tick,timestamp,tokenIn,tokenOut,...,traderWithdrawCount,traderSwapCount,ethAmount,usdAmount,price,fees,usd,eth,profit,datetime
0,6000000,2000000000,28963484807,28963.484807,15916883943069540192,28948.125166,-201261,1684801307,USDC,WETH,...,60818,60818,15.916884,28948.125166,1818.70555,14.481742,171036.515193,115.916884,-15.359641,2023-05-23 00:21:47+00:00
1,6000000,2000000000,26561134416,26561.134416,14592424671772653034,26544.819975,-201258,1684801781,USDC,WETH,...,60818,60818,14.592425,26544.819975,1819.08220,13.280567,144475.380777,130.509309,-25.678987,2023-05-23 00:29:41+00:00
2,6000000,2000000000,23734455145,23734.455145,13026029637798678527,23721.125260,-201248,1684802011,USDC,WETH,...,60818,60818,13.026030,23721.125260,1821.05568,11.867228,120740.925632,143.535338,21.200638,2023-05-23 00:33:31+00:00
3,6000000,2000000000,37540012512,37540.012512,20597925075725121430,37518.016271,-201246,1684802117,USDC,WETH,...,60818,60818,20.597925,37518.016271,1821.44639,18.770006,83200.913120,164.133263,16.214088,2023-05-23 00:35:17+00:00
4,6000000,2000000000,14727537731424333731,26848.252389,26833703476,26833.703476,-201239,1684802251,WETH,USDC,...,60818,60818,14.727538,26848.252389,1822.99668,13.424126,110034.616596,149.405726,101.090332,2023-05-23 00:37:31+00:00


In [38]:
TICK_BASE = 1.0001

def tick_to_price(tick):
    return TICK_BASE ** tick * 10 ** 12

user_df["tick_price"] = user_df["tick"].apply(tick_to_price)
user_df["price_diff"] = user_df["price"] - user_df["tick_price"]
user_df.head()

,gasLimit,gasPrice,amountIn,amountInUSD,amountOut,amountOutUSD,tick,timestamp,tokenIn,tokenOut,...,ethAmount,usdAmount,price,fees,usd,eth,profit,datetime,tick_price,price_diff
0,6000000,2000000000,28963484807,28963.484807,15916883943069540192,28948.125166,-201261,1684801307,USDC,WETH,...,15.916884,28948.125166,1818.70555,14.481742,171036.515193,115.916884,-15.359641,2023-05-23 00:21:47+00:00,1818.791146,-0.085596
1,6000000,2000000000,26561134416,26561.134416,14592424671772653034,26544.819975,-201258,1684801781,USDC,WETH,...,14.592425,26544.819975,1819.08220,13.280567,144475.380777,130.509309,-25.678987,2023-05-23 00:29:41+00:00,1819.336838,-0.254638
2,6000000,2000000000,23734455145,23734.455145,13026029637798678527,23721.125260,-201248,1684802011,USDC,WETH,...,13.026030,23721.125260,1821.05568,11.867228,120740.925632,143.535338,21.200638,2023-05-23 00:33:31+00:00,1821.156994,-0.101314
3,6000000,2000000000,37540012512,37540.012512,20597925075725121430,37518.016271,-201246,1684802117,USDC,WETH,...,20.597925,37518.016271,1821.44639,18.770006,83200.913120,164.133263,16.214088,2023-05-23 00:35:17+00:00,1821.521243,-0.074853
4,6000000,2000000000,14727537731424333731,26848.252389,26833703476,26833.703476,-201239,1684802251,WETH,USDC,...,14.727538,26848.252389,1822.99668,13.424126,110034.616596,149.405726,101.090332,2023-05-23 00:37:31+00:00,1822.796691,0.199989


In [50]:
df1 = user_df[['datetime', 'tokenIn', 'tokenOut', 'price_diff', 'price', 'tick_price', 'tick', 'fees', 'ethAmount', 'usdAmount', 'amountIn', 'amountInUSD', 'amountOut', 'amountOutUSD']]
df1.tail(100)

,datetime,tokenIn,tokenOut,price_diff,price,tick_price,tick,fees,ethAmount,usdAmount,amountIn,amountInUSD,amountOut,amountOutUSD
1889,2023-05-29 16:23:44+00:00,USDC,WETH,-0.131769,1879.86568,1879.997449,-200930,9.258459,9.844258,18505.882548,18516917865,18516.917865,9844257887848316903,18505.882548
1890,2023-05-29 16:30:42+00:00,USDC,WETH,0.892668,1882.20651,1881.313842,-200923,37.552160,39.905151,75109.734526,75104319036,75104.319036,39905150750832150191,75109.734526
1891,2023-05-29 16:33:02+00:00,USDC,WETH,1.456878,1882.77072,1881.313842,-200923,24.515261,26.050808,49047.697742,49030521850,49030.521850,26050807578801925576,49047.697742
1892,2023-05-29 16:34:52+00:00,WETH,USDC,0.215751,1884.35354,1884.137789,-200908,8.519103,9.041937,17038.205955,9041936979079760863,17038.205955,17029227735,17029.227735
1893,2023-05-29 16:35:08+00:00,WETH,USDC,0.208376,1884.15777,1883.949394,-200909,11.493151,12.199776,22986.301814,12199775507009482907,22986.301814,22973972977,22973.972977
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1984,2023-05-29 23:25:55+00:00,USDC,WETH,-0.076002,1898.05542,1898.131422,-200834,10.111330,10.648732,20211.883372,20222659128,20222.659128,10648731938686970574,20211.883372
1985,2023-05-29 23:30:17+00:00,WETH,USDC,0.129119,1896.93238,1896.803261,-200841,10.261879,10.819447,20523.758577,10819446593591710086,20523.758577,20512827563,20512.827563
1986,2023-05-29 23:34:08+00:00,WETH,USDC,0.301616,1896.15676,1895.855144,-200846,8.260616,8.713010,16521.232904,8713010048799521868,16521.232904,16512412746,16512.412746
1987,2023-05-29 23:41:02+00:00,WETH,USDC,0.209019,1895.30601,1895.096991,-200850,8.019196,8.462165,16038.391931,8462164867286618791,16038.391931,16029451162,16029.451162


In [57]:
print(tick_to_price(-201222))
print(tick_to_price(-201223))
print(tick_to_price(-201224))

1825.8979252658962
1825.7153537305232
1825.5328004504781


In [53]:
1818.791146 * 15.916884 - 28963.484807

-13.997115890935675

In [56]:
df1.sort_values(by=["price_diff"], ascending=False, inplace=True)
df1.tail(50)

/var/folders/sj/9n8j6m2s6_1g9x78vzd8z0rm0000gn/T/ipykernel_74522/2734388706.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.sort_values(by=["price_diff"], ascending=False, inplace=True)


,datetime,tokenIn,tokenOut,price_diff,price,tick_price,tick,fees,ethAmount,usdAmount,amountIn,amountInUSD,amountOut,amountOutUSD
1099,2023-05-27 18:33:16+00:00,USDC,WETH,-0.448974,1825.26638,1825.715354,-201223,18.045734,19.757587,36062.858987,36091467777,36091.467777,19757586828063619636,36062.858987
1056,2023-05-27 12:42:54+00:00,USDC,WETH,-0.462338,1829.27381,1829.736148,-201201,24.011561,26.234041,47989.244507,48023121743,48023.121743,26234041205179711422,47989.244507
1864,2023-05-29 16:02:02+00:00,USDC,WETH,-0.492066,1887.98401,1888.476076,-200885,9.020196,9.547692,18025.889566,18040392063,18040.392063,9547691860999754312,18025.889566
395,2023-05-24 19:19:01+00:00,USDC,WETH,-0.520098,1789.40339,1789.923488,-201421,7.654852,8.549221,15298.004274,15309703645,15309.703645,8549220572595912529,15298.004274
1867,2023-05-29 16:02:11+00:00,USDC,WETH,-0.521114,1890.22238,1890.743494,-200873,15.381656,16.262099,30738.983872,30763311122,30763.311122,16262099209632945005,30738.983872
1456,2023-05-28 22:02:35+00:00,USDC,WETH,-0.541478,1909.77633,1910.317808,-200770,6.380514,6.676587,12750.788159,12761028641,12761.028641,6676587178902931984,12750.788159
757,2023-05-26 00:21:16+00:00,USDC,WETH,-0.577542,1806.06904,1806.646582,-201328,15.915764,17.610402,31805.601565,31831527100,31831.527100,17610401851206712276,31805.601565
79,2023-05-23 03:20:23+00:00,WETH,USDC,-0.580343,1855.69329,1856.273633,-201057,12.694075,13.681221,25388.149167,13681220546189210352,25388.149167,25384977055,25384.977055
64,2023-05-23 03:10:19+00:00,USDC,WETH,-0.583458,1842.92661,1843.510068,-201126,40.046223,43.430073,80038.437965,80092445087,80092.445087,43430073412002240846,80038.437965
1353,2023-05-28 18:22:25+00:00,USDC,WETH,-0.587939,1852.90351,1853.491449,-201072,9.401001,10.138987,18786.564347,18802002305,18802.002305,10138986863249919355,18786.564347


In [58]:
df1.describe()

,price_diff,price,tick_price,tick,fees,ethAmount,usdAmount,amountInUSD,amountOutUSD
count,1989.000000,1989.000000,1989.000000,1989.000000,1989.000000,1989.000000,1989.000000,1989.000000,1989.000000
mean,0.078564,1847.277594,1847.199031,-201108.282051,11.711521,12.699067,23416.416640,23423.041771,23410.320329
std,0.315116,39.471342,39.464781,213.125053,5.497205,5.954771,10992.009483,10994.409014,10994.056668
min,-2.696039,1775.703220,1775.484387,-201502.000000,4.296618,4.503175,8593.235900,8593.235900,8588.750173
25%,-0.076946,1812.768560,1812.799327,-201294.000000,8.642977,9.373606,17276.817099,17285.954106,17276.817099
50%,0.105356,1845.772580,1845.723497,-201114.000000,10.386435,11.316145,20772.869034,20772.869034,20761.535598
75%,0.254053,1887.326380,1887.154672,-200892.000000,13.704746,14.887070,27404.804515,27409.492563,27394.756910
max,2.098042,1922.495680,1922.197975,-200708.000000,92.015935,96.089115,184031.869468,184031.869468,184252.467624


In [16]:
# trading account：0x8cc02c2381b7c55e18dccfea917f0677a5671931 analysis
# fetch all history transactions
import pandas as pd

from utils import utils

# new resolved USDT related swaps
df = pd.read_csv("../data/user_arb_swap_new.csv")
df["datetime"] = df["timestamp"].apply(utils.utc_timestamp_to_date)
df.head()

,id,amountIn,amountInUSD,amountOut,amountOutUSD,gasLimit,gasPrice,tick,timestamp,pool,tokenIn,tokenOut,datetime
0,0x4163f1437cefc0b6c73e00c8db819ea92aa07395a1a6...,3516117741730628903,5174.514646,5170663639,5170.663639,3000000,2000000000,-203382,1676306229,0xc31e54c7a869b9fcbecc14363cf510d1c41fa443,WETH,USDC,2023-02-13 16:37:09+00:00
1,0x058d12a79ee20e82379e5d0b2b1eeeaaa2988768244d...,4186892251423356044,6158.127305,6154400504,6154.400504,3000000,2000000000,-203387,1676306236,0xc31e54c7a869b9fcbecc14363cf510d1c41fa443,WETH,USDC,2023-02-13 16:37:16+00:00
2,0xcd8af3e9679b087671fa11627d3f0864b3bedb46523c...,3559441400205692687,5231.842059,5228940013,5228.940013,3000000,2000000000,-203393,1676306246,0xc31e54c7a869b9fcbecc14363cf510d1c41fa443,WETH,USDC,2023-02-13 16:37:26+00:00
3,0xa9ac38c9ce148d8830f73bec697d014de0ad3aa7e3a3...,8250801903,8250.801903,5613510956742205013,8245.020033,3000000,2000000000,-203396,1676306255,0xc31e54c7a869b9fcbecc14363cf510d1c41fa443,USDC,WETH,2023-02-13 16:37:35+00:00
4,0xc6ea270acc5833c393000dad84ffb0baaf793238f669...,3454446749687345942,5076.537527,5073730688,5073.730688,3000000,2000000000,-203395,1676306282,0xc31e54c7a869b9fcbecc14363cf510d1c41fa443,WETH,USDC,2023-02-13 16:38:02+00:00


In [17]:
df["pool"].value_counts()

0xc31e54c7a869b9fcbecc14363cf510d1c41fa443    62309
0x641c00a822e8b671738d32a431a4fb6074e5c79d        2
Name: pool, dtype: int64

In [18]:
df[df["pool"] == "0x641c00a822e8b671738d32a431a4fb6074e5c79d"].head()

,id,amountIn,amountInUSD,amountOut,amountOutUSD,gasLimit,gasPrice,tick,timestamp,pool,tokenIn,tokenOut,datetime
19209,0xdd6ccb08bb41d437daacff0faf7377f41fb247a855f2...,12500000000,12500.0,7779323090643218602,12497.533889,2661579,100000000,-202501,1678667866,0x641c00a822e8b671738d32a431a4fb6074e5c79d,USDC,WETH,2023-03-13 00:37:46+00:00
19213,0xa4bdb0ccdd3eb48c00d764fd29a1b2a42289769a460c...,12500000000,12500.0,7658940883836291687,12495.364145,2025922,100000000,-202347,1678668534,0x641c00a822e8b671738d32a431a4fb6074e5c79d,USDC,WETH,2023-03-13 00:48:54+00:00


In [19]:
df["tokenIn"].value_counts()

WETH    31589
USDC    30722
Name: tokenIn, dtype: int64

In [20]:
init_eth = 100
init_usd = 100 * 2000

# calculate eth, usd in every step to know least investment, considering about trading fee
trade_data = df.to_dict("records")

last_usd = init_usd
last_eth = init_eth

current_usd = 0
current_eth = 0
for item in trade_data:
    fees = item["amountInUSD"] * 0.0005
    if item["tokenIn"] == "USDC":
        current_usd = last_usd - item["amountInUSD"]
        current_eth = last_eth + int(item["amountOut"]) / (10 ** 18)
    if item["tokenIn"] == "WETH":
        current_usd = last_usd + item["amountOutUSD"]
        current_eth = last_eth - int(item["amountIn"]) / (10 ** 18)
    # profit = item["price"] * current_eth + current_usd - item["price"] * init_eth - init_usd
    # item["profit"] = profit
    item["fees"] = fees
    item["usd"] = current_usd
    item["eth"] = current_eth
    last_usd = current_usd
    last_eth = current_eth

user_df = pd.DataFrame(trade_data)
user_df.head()

,id,amountIn,amountInUSD,amountOut,amountOutUSD,gasLimit,gasPrice,tick,timestamp,pool,tokenIn,tokenOut,datetime,fees,usd,eth
0,0x4163f1437cefc0b6c73e00c8db819ea92aa07395a1a6...,3516117741730628903,5174.514646,5170663639,5170.663639,3000000,2000000000,-203382,1676306229,0xc31e54c7a869b9fcbecc14363cf510d1c41fa443,WETH,USDC,2023-02-13 16:37:09+00:00,2.587257,205170.663639,96.483882
1,0x058d12a79ee20e82379e5d0b2b1eeeaaa2988768244d...,4186892251423356044,6158.127305,6154400504,6154.400504,3000000,2000000000,-203387,1676306236,0xc31e54c7a869b9fcbecc14363cf510d1c41fa443,WETH,USDC,2023-02-13 16:37:16+00:00,3.079064,211325.064143,92.296990
2,0xcd8af3e9679b087671fa11627d3f0864b3bedb46523c...,3559441400205692687,5231.842059,5228940013,5228.940013,3000000,2000000000,-203393,1676306246,0xc31e54c7a869b9fcbecc14363cf510d1c41fa443,WETH,USDC,2023-02-13 16:37:26+00:00,2.615921,216554.004156,88.737549
3,0xa9ac38c9ce148d8830f73bec697d014de0ad3aa7e3a3...,8250801903,8250.801903,5613510956742205013,8245.020033,3000000,2000000000,-203396,1676306255,0xc31e54c7a869b9fcbecc14363cf510d1c41fa443,USDC,WETH,2023-02-13 16:37:35+00:00,4.125401,208303.202253,94.351060
4,0xc6ea270acc5833c393000dad84ffb0baaf793238f669...,3454446749687345942,5076.537527,5073730688,5073.730688,3000000,2000000000,-203395,1676306282,0xc31e54c7a869b9fcbecc14363cf510d1c41fa443,WETH,USDC,2023-02-13 16:38:02+00:00,2.538269,213376.932941,90.896613


In [21]:
user_df.describe()

,amountInUSD,amountOutUSD,gasLimit,gasPrice,tick,timestamp,fees,usd,eth
count,62311.000000,62311.000000,6.231100e+04,6.231100e+04,62311.000000,6.231100e+04,62311.000000,62311.000000,62311.000000
mean,20741.783744,20729.774617,5.766399e+06,2.001809e+09,-201595.445058,1.680365e+09,10.370892,221554.571869,135.054394
std,11747.459989,11742.637716,8.051630e+05,1.918357e+08,834.686582,2.668677e+06,5.873730,68134.194920,39.587890
min,0.003364,0.003361,3.000000e+05,1.000000e+08,-204072.000000,1.676306e+09,0.000002,69418.563161,12.498704
25%,13496.230969,13487.898459,6.000000e+06,2.000000e+09,-202161.000000,1.678199e+09,6.748115,170308.514337,106.720236
50%,17876.863275,17865.889427,6.000000e+06,2.000000e+09,-201418.000000,1.679821e+09,8.938432,220335.625334,134.247495
75%,24567.306635,24552.351235,6.000000e+06,2.000000e+09,-201007.000000,1.682683e+09,12.283653,269196.822368,163.910329
max,184031.869468,184252.467624,6.000000e+06,2.000000e+10,-199646.000000,1.685577e+09,92.015935,434449.527815,263.865053


In [22]:
user_df.tail(20)

,id,amountIn,amountInUSD,amountOut,amountOutUSD,gasLimit,gasPrice,tick,timestamp,pool,tokenIn,tokenOut,datetime,fees,usd,eth
62291,0x0c3114f3e68a0515ba5a23e30da85ed67404f7f88cc9...,10334526232439795675,19263.513389,19252810588,19252.810588,6000000,2000000000,-201017,1685568436,0xc31e54c7a869b9fcbecc14363cf510d1c41fa443,WETH,USDC,2023-05-31 21:27:16+00:00,9.631757,359792.199916,109.531302
62292,0x8ee7abcbae42cba1414e8bf94c60e0a60c28cf290324...,31549961239,31549.961239,16924679243087051699,31532.317262,6000000,2000000000,-201020,1685569163,0xc31e54c7a869b9fcbecc14363cf510d1c41fa443,USDC,WETH,2023-05-31 21:39:23+00:00,15.774981,328242.238677,126.455981
62293,0xcd11daeafba4aa3214927554bb202d6b52f86e8d8852...,24175842391,24175.842391,12958921098268170052,24162.954432,6000000,2000000000,-201012,1685570516,0xc31e54c7a869b9fcbecc14363cf510d1c41fa443,USDC,WETH,2023-05-31 22:01:56+00:00,12.087921,304066.396286,139.414902
62294,0x0474edfc2daf02e8137436b385c48ab19d213c12ed7a...,29146440454,29146.440454,15618879921438869991,29130.033231,6000000,2000000000,-201009,1685570896,0xc31e54c7a869b9fcbecc14363cf510d1c41fa443,USDC,WETH,2023-05-31 22:08:16+00:00,14.573220,274919.955832,155.033782
62295,0x58c13fc22202e305adf924035539543e65afa13bd083...,27785022304137915750,51836.806223,51864191602,51864.191602,6000000,2000000000,-200998,1685571346,0xc31e54c7a869b9fcbecc14363cf510d1c41fa443,WETH,USDC,2023-05-31 22:15:46+00:00,25.918403,326784.147434,127.248760
62296,0x30dcc30285fe61476e1bef9b764f812460c798bd404a...,19672965528,19672.965528,10532991695467945066,19661.216300,6000000,2000000000,-201001,1685571566,0xc31e54c7a869b9fcbecc14363cf510d1c41fa443,USDC,WETH,2023-05-31 22:19:26+00:00,9.836483,307111.181906,137.781751
62297,0x2bb694d838b5e73838042aabe502e5bdd97a8d04f27b...,31714858836,31714.858836,16966626007516860900,31694.508937,6000000,2000000000,-200992,1685571586,0xc31e54c7a869b9fcbecc14363cf510d1c41fa443,USDC,WETH,2023-05-31 22:19:46+00:00,15.857429,275396.323070,154.748377
62298,0x4cc26cd39a96c3b9d8eae8ee5da4d9940ad860290873...,25215693051,25215.693051,13491238524117610285,25202.152706,6000000,2000000000,-200994,1685572753,0xc31e54c7a869b9fcbecc14363cf510d1c41fa443,USDC,WETH,2023-05-31 22:39:13+00:00,12.607847,250180.630019,168.239616
62299,0x68d98ae6335389f581b1aaec11f569a58c49f4a601b7...,17748684706,17748.684706,9491276168291560701,17736.639253,6000000,2000000000,-200989,1685573957,0xc31e54c7a869b9fcbecc14363cf510d1c41fa443,USDC,WETH,2023-05-31 22:59:17+00:00,8.874342,232431.945313,177.730892
62300,0xee24a7aeccdab37852bca614f7cbd832b3e449e933a0...,17222940830960084077,32209.634701,32188293419,32188.293419,6000000,2000000000,-200985,1685574011,0xc31e54c7a869b9fcbecc14363cf510d1c41fa443,WETH,USDC,2023-05-31 23:00:11+00:00,16.104817,264620.238732,160.507951


In [23]:
user_df.head(100)

,id,amountIn,amountInUSD,amountOut,amountOutUSD,gasLimit,gasPrice,tick,timestamp,pool,tokenIn,tokenOut,datetime,fees,usd,eth
0,0x4163f1437cefc0b6c73e00c8db819ea92aa07395a1a6...,3516117741730628903,5174.514646,5170663639,5170.663639,3000000,2000000000,-203382,1676306229,0xc31e54c7a869b9fcbecc14363cf510d1c41fa443,WETH,USDC,2023-02-13 16:37:09+00:00,2.587257,205170.663639,96.483882
1,0x058d12a79ee20e82379e5d0b2b1eeeaaa2988768244d...,4186892251423356044,6158.127305,6154400504,6154.400504,3000000,2000000000,-203387,1676306236,0xc31e54c7a869b9fcbecc14363cf510d1c41fa443,WETH,USDC,2023-02-13 16:37:16+00:00,3.079064,211325.064143,92.296990
2,0xcd8af3e9679b087671fa11627d3f0864b3bedb46523c...,3559441400205692687,5231.842059,5228940013,5228.940013,3000000,2000000000,-203393,1676306246,0xc31e54c7a869b9fcbecc14363cf510d1c41fa443,WETH,USDC,2023-02-13 16:37:26+00:00,2.615921,216554.004156,88.737549
3,0xa9ac38c9ce148d8830f73bec697d014de0ad3aa7e3a3...,8250801903,8250.801903,5613510956742205013,8245.020033,3000000,2000000000,-203396,1676306255,0xc31e54c7a869b9fcbecc14363cf510d1c41fa443,USDC,WETH,2023-02-13 16:37:35+00:00,4.125401,208303.202253,94.351060
4,0xc6ea270acc5833c393000dad84ffb0baaf793238f669...,3454446749687345942,5076.537527,5073730688,5073.730688,3000000,2000000000,-203395,1676306282,0xc31e54c7a869b9fcbecc14363cf510d1c41fa443,WETH,USDC,2023-02-13 16:38:02+00:00,2.538269,213376.932941,90.896613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0x579a3257f4501722eae007690a44cc59aee3ee897972...,6868417967,6868.417967,4659471665448937926,6864.542775,3000000,2000000000,-203367,1676309155,0xc31e54c7a869b9fcbecc14363cf510d1c41fa443,USDC,WETH,2023-02-13 17:25:55+00:00,3.434209,175382.016118,116.767420
96,0x8b980701e1a2e2579367caa93a0fa0b7fed2f2a8290e...,5732943680,5732.943680,3888074770402788980,5729.533040,3000000,2000000000,-203365,1676309164,0xc31e54c7a869b9fcbecc14363cf510d1c41fa443,USDC,WETH,2023-02-13 17:26:04+00:00,2.866472,169649.072438,120.655495
97,0xd53b0de578e8af866968eadd44cdca085219418952c6...,5624351415,5624.351415,3813495708227086284,5620.933575,3000000,2000000000,-203362,1676309169,0xc31e54c7a869b9fcbecc14363cf510d1c41fa443,USDC,WETH,2023-02-13 17:26:09+00:00,2.812176,164024.721023,124.468990
98,0x3764abe9ef5de3bcf3da5e2afc74dd6460c08569d571...,4713049368144335810,6948.387037,6944018692,6944.018692,3000000,2000000000,-203364,1676309251,0xc31e54c7a869b9fcbecc14363cf510d1c41fa443,WETH,USDC,2023-02-13 17:27:31+00:00,3.474194,170968.739715,119.755941


In [24]:
TICK_BASE = 1.0001

def tick_to_price(tick):
    return TICK_BASE ** tick * 10 ** 12

user_df["tick_price"] = user_df["tick"].apply(tick_to_price)
user_df.head()

,id,amountIn,amountInUSD,amountOut,amountOutUSD,gasLimit,gasPrice,tick,timestamp,pool,tokenIn,tokenOut,datetime,fees,usd,eth,tick_price
0,0x4163f1437cefc0b6c73e00c8db819ea92aa07395a1a6...,3516117741730628903,5174.514646,5170663639,5170.663639,3000000,2000000000,-203382,1676306229,0xc31e54c7a869b9fcbecc14363cf510d1c41fa443,WETH,USDC,2023-02-13 16:37:09+00:00,2.587257,205170.663639,96.483882,1471.206304
1,0x058d12a79ee20e82379e5d0b2b1eeeaaa2988768244d...,4186892251423356044,6158.127305,6154400504,6154.400504,3000000,2000000000,-203387,1676306236,0xc31e54c7a869b9fcbecc14363cf510d1c41fa443,WETH,USDC,2023-02-13 16:37:16+00:00,3.079064,211325.064143,92.296990,1470.470921
2,0xcd8af3e9679b087671fa11627d3f0864b3bedb46523c...,3559441400205692687,5231.842059,5228940013,5228.940013,3000000,2000000000,-203393,1676306246,0xc31e54c7a869b9fcbecc14363cf510d1c41fa443,WETH,USDC,2023-02-13 16:37:26+00:00,2.615921,216554.004156,88.737549,1469.588948
3,0xa9ac38c9ce148d8830f73bec697d014de0ad3aa7e3a3...,8250801903,8250.801903,5613510956742205013,8245.020033,3000000,2000000000,-203396,1676306255,0xc31e54c7a869b9fcbecc14363cf510d1c41fa443,USDC,WETH,2023-02-13 16:37:35+00:00,4.125401,208303.202253,94.351060,1469.148159
4,0xc6ea270acc5833c393000dad84ffb0baaf793238f669...,3454446749687345942,5076.537527,5073730688,5073.730688,3000000,2000000000,-203395,1676306282,0xc31e54c7a869b9fcbecc14363cf510d1c41fa443,WETH,USDC,2023-02-13 16:38:02+00:00,2.538269,213376.932941,90.896613,1469.295074


In [27]:
user_df[["datetime", "tokenIn", "tokenOut", "tick", "tick_price", "amountIn", "amountOut", "amountInUSD", "amountOutUSD", "usd", "eth"]].head(100)

,datetime,tokenIn,tokenOut,tick,tick_price,amountIn,amountOut,amountInUSD,amountOutUSD,usd,eth
0,2023-02-13 16:37:09+00:00,WETH,USDC,-203382,1471.206304,3516117741730628903,5170663639,5174.514646,5170.663639,205170.663639,96.483882
1,2023-02-13 16:37:16+00:00,WETH,USDC,-203387,1470.470921,4186892251423356044,6154400504,6158.127305,6154.400504,211325.064143,92.296990
2,2023-02-13 16:37:26+00:00,WETH,USDC,-203393,1469.588948,3559441400205692687,5228940013,5231.842059,5228.940013,216554.004156,88.737549
3,2023-02-13 16:37:35+00:00,USDC,WETH,-203396,1469.148159,8250801903,5613510956742205013,8250.801903,8245.020033,208303.202253,94.351060
4,2023-02-13 16:38:02+00:00,WETH,USDC,-203395,1469.295074,3454446749687345942,5073730688,5076.537527,5073.730688,213376.932941,90.896613
...,...,...,...,...,...,...,...,...,...,...,...
95,2023-02-13 17:25:55+00:00,USDC,WETH,-203367,1473.414659,6868417967,4659471665448937926,6868.417967,6864.542775,175382.016118,116.767420
96,2023-02-13 17:26:04+00:00,USDC,WETH,-203365,1473.709356,5732943680,3888074770402788980,5732.943680,5729.533040,169649.072438,120.655495
97,2023-02-13 17:26:09+00:00,USDC,WETH,-203362,1474.151513,5624351415,3813495708227086284,5624.351415,5620.933575,164024.721023,124.468990
98,2023-02-13 17:27:31+00:00,WETH,USDC,-203364,1473.856727,4713049368144335810,6944018692,6948.387037,6944.018692,170968.739715,119.755941


In [28]:
user_df.shape

(62311, 17)

In [29]:
62311 / 108 / 24

24.039737654320987

In [ ]:
# calculate buy price & sell price
# trade strategy analysis